In [1]:
import nltk as nt
import numpy as np
import pandas as pd

In [2]:
complete_array = ["CC","CD","DT","EX","IN","JJ","JJR","JJS","LS","MD","NN","NNP","NNS","PDT","POS","PRP","PRP$","RB","RBR","RBS","RP","TO","UH","VB","VBD","VBG","VBN","VBP","VBZ","WDT","WP","WRB"]
noun_phrase_array = ["NN","NNP","NNS","PRP","POS","WP"]
verb_phrase_array = ["RB","RBR","RBS","VB","VBD","VBG","VBN","VBP","VBZ","WRB"]
det_phrase_array = ["DT","PDT","WDT","PRP$"]

In [11]:
def sent_struct(string):
    sent = string
    try:
        a = nt.word_tokenize(sent)
        s = nt.pos_tag(a)
        return s
    except:
        return 'This string has a smiley!!'

In [12]:
def s_line(sent):
    words = sent.split()
    structs = sent_struct(sent)
    string = 'S -> '
    ncount,vcount = 0,0
    for struct in structs:
        for each in noun_phrase_array:
            if(struct[1] == each):
                ncount += 1
            else:
                continue
        for each in verb_phrase_array:
            if(struct[1] == each):
                vcount += 1
            else:
                continue
    if(ncount > 0 and vcount > 0):
        string = string + 'NP VP'
    elif(ncount > 0 and vcount == 0):
        string = string  + 'NP'
    elif(ncount == 0 and vcount > 0):
        string = string  + 'VP'
    elif(ncount == 0 and vcount == 0):
        string = None
    return string,ncount,vcount

In [13]:
def p_line(sent):
    words = sent.split()
    structs= sent_struct(sent)
    string = 'PP -> '
    pcount = 0
    for struct in structs:
        if(struct[1] == 'IN'):
            pcount += 1
        else:
            continue
    if(pcount > 0):
        string += 'P NP'
    else:
        string = None
    return string,pcount

In [14]:
def np_line(sent):
    words = sent.split()
    structs= sent_struct(sent)
    string = 'NP -> '
    dcount,icount = 0,0
    for struct in structs:
        for each in det_phrase_array:
            if(struct[1] == each):
                dcount += 1
            else:
                continue
        if(struct[0] == 'I' or struct[0] == 'i'):
            icount += 1
        else:
            continue
    if(dcount > 0 and icount > 0):
        string = string + """Det N | NP PP | 'I'"""
    elif(dcount > 0 and icount == 0):
        string = string  + 'Det N | NP PP'
    elif(dcount == 0 and icount > 0):
        string = string  + """'I'"""
    elif(dcount == 0 and icount == 0):
        string = None
    return string,dcount,icount

In [15]:
def vp_line(sent):
    pcount = p_line(sent)[1]
    temp_string = s_line(sent)
    string = 'VP -> '
    if(temp_string != None and pcount != 0):
        string += 'V NP | VP PP'
    elif(temp_string == None and pcount != 0):
        string += 'VP PP'
    elif(temp_string != None and pcount == 0):
        string += 'V NP'
    elif(temp_string == None and pcount == 0):
        string = None
    return string

In [16]:
def Det(sent):
    structs= sent_struct(sent)
    string = 'Det ->'
    for struct in structs:
        for each in det_phrase_array:
            if(struct[1] == each):
                string += " " + "'" + struct[0] + "'" + "|"
            else:
                continue
    return string.strip("|")

def N(sent):
    structs= sent_struct(sent)
    string = 'N ->'
    for struct in structs:
        for each in noun_phrase_array:
            if(struct[1] == each and struct[0] != 'I'):
                string += " " + "'" + struct[0] + "'" + "|"
            else:
                continue
    return string.strip("|")

def V(sent):
    structs= sent_struct(sent)
    string = 'V ->'
    for struct in structs:
        for each in verb_phrase_array:
            if(struct[1] == each):
                string += " " + "'" + struct[0] + "'" + "|"
            else:
                continue
    return string.strip("|")

def P(sent):
    structs= sent_struct(sent)
    string = 'P ->'
    for struct in structs:
        if(struct[1] == "IN"):
            string += " " + "'" + struct[0] + "'" + "|"
        else:
            continue
    return string.strip("|")
    

In [17]:
sent_struct(sent)

[('No', 'DT'),
 ('smoking', 'NN'),
 ('area', 'NN'),
 ('allow', 'VBP'),
 ('hookas', 'NN'),
 ('inside', 'IN')]

In [18]:
s_line(sent)[0],p_line(sent)[0],np_line(sent)[0],vp_line(sent)

('S -> NP VP', 'PP -> P NP', 'NP -> Det N | NP PP', 'VP -> V NP | VP PP')

In [19]:
Det(sent),N(sent),V(sent),P(sent)

("Det -> 'No'",
 "N -> 'smoking'| 'area'| 'hookas'",
 "V -> 'allow'",
 "P -> 'inside'")

In [20]:
def grammar(sent):
    return """
    {a}
    {b}
    {c}
    {d}
    {e}
    {f}
    {g}
    {h}
    """.format(
        a=s_line(sent)[0],b=p_line(sent)[0],c=np_line(sent)[0],d=vp_line(sent),e=Det(sent),f=N(sent),g=V(sent),h=P(sent))

In [21]:
grammar(sent)

"\n    S -> NP VP\n    PP -> P NP\n    NP -> Det N | NP PP\n    VP -> V NP | VP PP\n    Det -> 'No'\n    N -> 'smoking'| 'area'| 'hookas'\n    V -> 'allow'\n    P -> 'inside'\n    "

In [37]:
array_sent = sent1.split()
groucho_grammar = nt.CFG.fromstring(grammar(sent1))
parser = nt.ChartParser(groucho_grammar)
parser.parse(array_sent)
for tree in parser.parse(array_sent):
    print(tree)

(S
  (NP I)
  (VP
    (VP (V lifted) (NP (Det an) (N elephant)))
    (PP (P with) (NP (Det a) (N hand)))))
(S
  (NP I)
  (VP
    (V lifted)
    (NP
      (NP (Det an) (N elephant))
      (PP (P with) (NP (Det a) (N hand))))))


In [23]:
import nltk as nt
import numpy as np
import pandas as pd

a =[]
s =[]

def bag_of_words(path):
    csv = pd.read_csv(path,index_col=0)
    df = pd.DataFrame(csv)
    text_array = df.loc[:,'text']
    numpy_text_array = text_array.as_matrix()
    for i in range(len(numpy_text_array)):
        try:
            a = nt.word_tokenize(numpy_text_array[i])
            s += nt.pos_tag(a)
        except:
            continue;
    return s

In [24]:
path = "C:/Users/ramda/Desktop/csv changed.csv"

In [25]:
csv = pd.read_csv(path,index_col=0)

In [26]:
df = pd.DataFrame(csv)

In [27]:
text_array = df.loc[:,'text']

In [28]:
seen = set()
ans = []

In [29]:
for item in s:
    if item[1] not in seen:
        ans.append(item)
        seen.add(item[1])

In [30]:
nt.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [36]:
sent = 'No smoking area allow hookas inside'
#sent1 = 'I shot an elephant in my pajamas'
sent1 = 'I lifted an elephant with a hand'